In [1]:
# либы шаблона (нужны всегда)
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# либы текущего решения (нужны здесь)
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB

получаем данные

In [4]:
%%time
X = pd.read_csv('data/train.csv').drop('Times', 1)
y = pd.read_csv('data/train.csv')['Times']

MODE = 'validation'
if MODE == 'validation':
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
elif MODE == 'test':
    x_train = X
    y_train = y
    x_test = pd.read_csv('data/test.csv')
 

определяем метрику

In [31]:
def rmsle(y_pred, y_test) : 
    return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_test))**2))

### Решение

Определяем шаблоны паролей

In [7]:
patterns_match = [
    r'^[\d]{4}$', r'^[\d]{6}$', r'^[\d]{8}$', r'^[\d]{8,}$', r'^[\d]{4,8}$',
    r'^[\d]+$', r'^[a-z]+$', r'^[A-Z]', r'^[A-Z]+$', r'^[A-Z][a-z]',
    r'^[A-Za-z]+[0-9]+$', r'^[A-Za-z]+1$', r'^1[A-Za-z]+$', r'^[A-Z]+[0-9]+$', r'^[a-z]+[0-9]+$',
    r'^[0-9]+[a-z]+$', r'^[0-9]+[a-zA-Z]+$', r'^[a-z]+[0-9]{3,}$', r'^[a-z]+[0-9]$', r'^[a-z]+[0-9]{1,3}$', r'^[A-Za-z]+[0-9]+[A-Za-z]+', r'^[A-Za-z]+[0-9]+[A-Za-z]+[0-9]+',
    r'([\d]{2,})(\1)+', r'.*([\w]+)(\1)+.*'
    
]

Находим соответствие каждому шаблону

In [8]:
df_t = pd.DataFrame()
df_v = pd.DataFrame()
for pat in tqdm(patterns_match):
    df_t[str(pat)] = x_train['Password'].str.match(pat)
    df_v[str(pat)] = x_test['Password'].str.match(pat)

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [02:08<00:00,  5.37s/it]


In [18]:
df_t = df_t.fillna(0)

Вместо оригинального целевого признака, используем бинарный (больше 1 или нет)

In [9]:
y_train_bin = y_train > 1

Обучаем NaiveBayes и предсказываем бинарный целевой признак для теста

In [11]:
nb = BernoulliNB()

In [19]:
# качество ни к черту, попробуем дальше
cross_val_score(nb, df_t, y_train_bin, cv=4, scoring='precision')

array([0.34037206, 0.34151768, 0.34253936, 0.34883956])

In [21]:
nb.fit(df_t, y_train_bin)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [24]:
y_preds_train_bin = nb.predict(df_t)
y_preds_bin = nb.predict(df_v)

Для каждого предсказанного класса находим оптимальное константное решение

In [27]:
c1 = np.exp(np.log(y_train[y_preds_train_bin] + 1).mean()) - 1
c1

2.025422289268158

In [28]:
c2 = np.exp(np.log(y_train[~y_preds_train_bin] + 1).mean()) - 1
c2

1.2171156525958042

Создаем решение: для теста предсказываем бинарный целевой признак, и в соответсвии с ним выдаем оптимальную константу (c1 или c2)

In [34]:
c1 = np.exp(np.log(y_train[y_preds_train_bin] + 1).mean()) - 1
c2 = np.exp(np.log(y_train[~y_preds_train_bin] + 1).mean()) - 1
ypred = pd.Series(c2, index=y_test.index)
ypred[y_preds_bin] = c1

if MODE == 'validation':
    print(rmsle(y_test, ypred))
elif MODE == 'test':
    ypred.rename('Times').reset_index().rename({'index': 'Id'}, axis=1).to_csv('submit_trial_next.csv', index=False)

0.401423028644326
